In [9]:
!pip install gmaps


SyntaxError: invalid syntax (Temp/ipykernel_17672/643705268.py, line 2)

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data = pd.read_csv("output_data/cities.csv")
city_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,58.75,68.0,64.0,3.36,ZA,1.650866e+09
1,1,wattegama,6.7989,81.4808,92.16,51.0,60.0,1.88,LK,1.650866e+09
2,2,half moon bay,37.4636,-122.4286,56.48,77.0,0.0,16.11,US,1.650866e+09
3,3,yellowknife,62.4560,-114.3525,30.72,69.0,20.0,13.80,CA,1.650866e+09
4,4,avarua,-21.2078,-159.7750,77.05,88.0,98.0,1.14,CK,1.650866e+09
...,...,...,...,...,...,...,...,...,...,...
546,597,guatire,10.4762,-66.5427,76.69,96.0,80.0,1.34,VE,1.650866e+09
547,598,malakal,9.5334,31.6605,84.83,55.0,73.0,6.33,SS,1.650866e+09
548,599,saiha,22.4833,92.9667,96.15,24.0,0.0,5.26,IN,1.650866e+09
549,600,atar,20.5169,-13.0499,71.28,38.0,0.0,7.43,MR,1.650866e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#gmaps.configure(api_keys=g_key)

humid = city_data["Humidity"].astype(float)
maxhum = humid.max()
locs = city_data[["Lat", "Lng"]]

In [6]:
hum_fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locs, weights=humid,dissipating=False, max_intensity=maxhum,point_radius=3)
hum_fig.add_layer(heat_layer)
hum_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
narrowed_city_df = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 80) & (city_data["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,37,tessalit,20.1986,1.0114,72.07,20.0,0.0,8.46,ML,1.650866e+09
1,71,araouane,18.9048,-3.5265,72.14,30.0,0.0,10.65,ML,1.650866e+09
2,103,virginia beach,36.8529,-75.9780,70.02,79.0,0.0,3.44,US,1.650866e+09
3,104,xai-xai,-25.0519,33.6442,74.50,79.0,0.0,0.45,MZ,1.650866e+09
4,192,abu samrah,35.3029,37.1841,76.75,27.0,0.0,7.52,SY,1.650866e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotels = []

for x in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[x]['Lat']
    lng = narrowed_city_df.loc[x]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
narrowed_city_df["Hotel Name"] = hotels
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,37,tessalit,20.1986,1.0114,72.07,20.0,0.0,8.46,ML,1.650866e+09,Achemelmel
1,71,araouane,18.9048,-3.5265,72.14,30.0,0.0,10.65,ML,1.650866e+09,Araouane
2,103,virginia beach,36.8529,-75.9780,70.02,79.0,0.0,3.44,US,1.650866e+09,Virginia Beach
3,104,xai-xai,-25.0519,33.6442,74.50,79.0,0.0,0.45,MZ,1.650866e+09,Xai-Xai District
4,192,abu samrah,35.3029,37.1841,76.75,27.0,0.0,7.52,SY,1.650866e+09,Suruj


In [2]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig